# 🏆 Rank 1 Solution — NeuralNet_Nexus_2.0

Hi everyone,  
I am **Athul Krishna Girish**, and this notebook presents my **Rank 1 solution** for **NeuralNet_Nexus_2.0**.

**Leaderboard Scores**
- **Public Score:** 0.90445  
- **Private Score:** 0.91497  

---

## 📌 Experience & Learning

This was a fantastic competition, and as a beginner, I learned a great deal throughout this journey. The problem encouraged deep experimentation, feature engineering, and careful validation strategies.

---

## ⚠️ Important Clarification on Leaderboard Scores

You may notice that I currently have a **0.99 score on the leaderboard**. This score was the result of **data leakage**, and I would like to clarify this transparently.

The dataset used in this competition is **publicly available online**. From prior experience, searching for similar datasets is a common, legitimate and always recommeded practice. However, in this case, the dataset I found **coincidently** happened to be **the exact dataset that was split into train and test sets for this competition**, which resulted in a near-perfect score.

🔹 **Please note:**  
- I **explicitly unticked** that submission as it was influenced by data leakage.  
- Unfortunately, I forgot to tick another valid submission as a beginner, due to which Kaggle automatically selected the best-performing one.

---

## ✅ The Actual Rank 1 Submission

The solution presented in **this notebook**:
- Uses **only the competition-provided data**
- Uses **no external data**
- Uses **no pretrained models**
- Achieved **0.91497 on the private leaderboard**
- Secured **Rank 1 legitimately**

This notebook:
1. Trains the final model entirely from scratch  
2. Runs inference on `test.csv`  
3. Generates `submission.csv`  
4. Can be fully replicated  
5. Produces matching public and private leaderboard scores upon submission  

---

## 🧠 Solution Overview

### 🔹 Feature Engineering
- Extensive feature engineering was performed
- In hindsight, ~25% of features did not contribute significantly and could have been discarded

### 🔹 Model Choice
- **XGBoost** was used exclusively
- No ensemble methods were required
- The final submission was made using **a single XGBoost model trained on the full (training) dataset**

---

## 🔍 Choosing `n_estimators` Without a Validation Set

Since the final model was trained on **100% of the data**, traditional early stopping using a validation set was not possible.

So was `n_estimators` chosen by trial and error?  
**No.**

### Strategy Used:
1. Performed **5-fold Stratified Cross-Validation**
2. Trained the same XGBoost configuration on each fold
3. Recorded the number of trees where early stopping occurred in each fold
4. Computed the **mean `n_estimators`** across the 5 folds  
   - Note: This mean corresponds to training on **80% of the data**
5. Multiplied the mean by **1.15**  
   - Reason: Training on **100% of the data** requires more trees than training on 80%

This approach provides a principled estimate for `n_estimators` when training on the full dataset.

📖 **Credit:**  
This idea was originally proposed by **Chris Deotte (NVIDIA)** in a Rank 1 solution for a similar Kaggle Playground Series competition.

---

## Version Discrepancy
Note that the version of each library used locally were as follows:

Library Versions:
- Pandas version: 2.2.3
- numpy version: 2.1.3
- scikit-learn version: 1.7.2
- xgboost version: 3.1.2

Since these version don't match with the Kaggle ones, you might see a different result. For example, in my kaggle run I also got a score of 0.91659 which actually beats my local score. However, if you corretly set the version of each package as mentioned above, you will get identical public and private scores that are mentioned at the top of this cell.

---

## 💬 Final Notes

If you have any questions or doubts, please feel free to comment on this notebook. I will be happy to respond.

A big thank you to **Govt. Model Engineering College, Thrikkakkara** for organizing this engaging competition. I am looking forward to many more such events in the future.

Also, congratulations to **Team Visionaries (Sreeda Sreekanth & K Darshini)** for topping the leaderboard as well — we both achieved the exact same scores.

**Thank you!**


## 1. Import Libraries

In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
import xgboost
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [5]:
print("Library Versions:")
print("=" * 50)

# Expected versions
expected_versions = {
    'pandas': '2.2.3',
    'numpy': '2.1.3',
    'scikit-learn': '1.7.2',
    'xgboost': '3.1.2'
}

# Actual versions
actual_versions = {
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'scikit-learn': sklearn.__version__,
    'xgboost': xgboost.__version__
}

# Compare and display
for lib, expected in expected_versions.items():
    actual = actual_versions[lib]
    match = "✓" if actual == expected else "✗"
    print(f"{match} {lib:20s} Local: {expected:10s} | Kaggle: {actual:10s}")


Library Versions:
✓ pandas               Local: 2.2.3      | Kaggle: 2.2.3     
✗ numpy                Local: 2.1.3      | Kaggle: 1.26.4    
✗ scikit-learn         Local: 1.7.2      | Kaggle: 1.2.2     
✗ xgboost              Local: 3.1.2      | Kaggle: 2.0.3     


## 2. Load Data

In [6]:
# Load training and test data
train_df = pd.read_csv('/kaggle/input/neural-net-nexus-2-0/train.csv')
test_df = pd.read_csv('/kaggle/input/neural-net-nexus-2-0/test.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTrain columns: {train_df.columns.tolist()}")
print(f"\nTrain data types:\n{train_df.dtypes}")

Train shape: (9880, 18)
Test shape: (2470, 18)

Train columns: ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']

Train data types:
Administrative             float64
Administrative_Duration    float64
Informational              float64
Informational_Duration     float64
ProductRelated             float64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems           float64
Browser                    float64
Region                     float64
TrafficType                float64
VisitorType                 object
Weekend                     object
Revenue                       b

## 3. Explore Data

In [7]:
# Check for missing values
print("Missing values in train:")
print(train_df.isnull().sum())
print("\nMissing values in test:")
print(test_df.isnull().sum())

# Check target distribution
print("\nRevenue distribution:")
print(train_df['Revenue'].value_counts())

Missing values in train:
Administrative             195
Administrative_Duration    195
Informational              195
Informational_Duration     195
ProductRelated             195
ProductRelated_Duration    195
BounceRates                195
ExitRates                  195
PageValues                 195
SpecialDay                 195
Month                      195
OperatingSystems           195
Browser                    195
Region                     195
TrafficType                195
VisitorType                195
Weekend                    195
Revenue                      0
dtype: int64

Missing values in test:
ID                          0
Administrative             52
Administrative_Duration    52
Informational              52
Informational_Duration     52
ProductRelated             52
ProductRelated_Duration    52
BounceRates                52
ExitRates                  52
PageValues                 52
SpecialDay                 52
Month                      52
OperatingSystems   

## 4. Identify Categorical and Numerical Columns

In [8]:
# Separate features and target
X_train = train_df.drop('Revenue', axis=1)
y_train = train_df['Revenue']

# Store test IDs for submission
test_ids = test_df['ID'].copy()
X_test = test_df.drop('ID', axis=1)

# Identify categorical columns (object dtype or specific columns)
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"\nNumerical columns: {numerical_cols}")

Categorical columns: ['Month', 'VisitorType', 'Weekend']

Numerical columns: ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'OperatingSystems', 'Browser', 'Region', 'TrafficType']


## 4a. Handle Missing Values

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Check for missing values before imputation
print("Missing values BEFORE imputation:")
print("\nTrain set:")
train_missing = train_df.isnull().sum()
print(train_missing[train_missing > 0] if train_missing.sum() > 0 else "No missing values")

print("\nTest set:")
test_missing = test_df.isnull().sum()
print(test_missing[test_missing > 0] if test_missing.sum() > 0 else "No missing values")

# Prepare data for imputation (X_train and X_test without target)
X_train = train_df.drop('Revenue', axis=1)
y_train = train_df['Revenue']
test_ids = test_df['ID'].copy()
X_test = test_df.drop('ID', axis=1)

# Separate categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# STEP 1: Encode categorical features BEFORE imputation
print(f"\n{'='*60}")
print("Step 1: Encoding categorical features for imputation")
print(f"{'='*60}")

X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    # Fit on combined data to ensure consistency
    combined = pd.concat([X_train_encoded[col], X_test_encoded[col]], axis=0)
    le.fit(combined)
    
    # Transform both datasets
    X_train_encoded[col] = le.transform(X_train_encoded[col])
    X_test_encoded[col] = le.transform(X_test_encoded[col])
    label_encoders[col] = le
    
    print(f"  ✓ Encoded {col}")

print(f"✓ All categorical features encoded to numerical values")

# STEP 2: COMBINE train and test data for imputation (CRITICAL!)
print(f"\n{'='*60}")
print("Step 2: Combining train+test for imputation training")
print(f"{'='*60}")
print("This ensures the imputer sees the full distribution of both datasets")

# Combine both datasets temporarily for imputation
X_combined = pd.concat([X_train_encoded, X_test_encoded], axis=0, ignore_index=True)
print(f"Combined shape: {X_combined.shape}")
print(f"Train samples: {len(X_train_encoded)}, Test samples: {len(X_test_encoded)}")

# STEP 3: Apply IterativeImputer to COMBINED data
print(f"\n{'='*60}")
print("Step 3: Fitting IterativeImputer on combined train+test data")
print(f"{'='*60}")
print("This models each feature as a function of other features")
print("using the full distribution from both datasets...")

# Apply IterativeImputer to combined data
iterative_imputer = IterativeImputer(
    estimator=None,  # Uses BayesianRidge by default
    max_iter=10,  # Number of iterations
    random_state=42,
    verbose=0
)

print(f"\nFitting imputer on combined data...")
X_combined_imputed = iterative_imputer.fit_transform(X_combined)

# Convert back to DataFrame with original column names
X_combined_imputed_df = pd.DataFrame(X_combined_imputed, columns=X_combined.columns)

# Split back into train and test
X_train = X_combined_imputed_df.iloc[:len(X_train_encoded)].reset_index(drop=True)
X_test = X_combined_imputed_df.iloc[len(X_train_encoded):].reset_index(drop=True)

print(f"✓ Imputation complete using combined dataset!")

# STEP 4: Convert categorical columns back to integers (required for CatBoost)
print(f"\n{'='*60}")
print("Step 4: Converting categorical columns to integer dtype")
print(f"{'='*60}")
for col in categorical_cols:
    X_train[col] = X_train[col].round().astype('int64')
    X_test[col] = X_test[col].round().astype('int64')
    print(f"  ✓ {col} converted to int64")

print(f"✓ Iterative imputation complete!")

# Verify no missing values remain
print(f"\n{'='*60}")
print("Missing values AFTER imputation:")
print(f"{'='*60}")
train_missing_after = X_train.isnull().sum().sum()
test_missing_after = X_test.isnull().sum().sum()
print(f"Train set: {train_missing_after} missing values")
print(f"Test set: {test_missing_after} missing values")
print(f"✓ All missing values handled successfully!")
print(f"\nData types after imputation:")
print(f"Categorical columns are now: {X_train[categorical_cols].dtypes.unique()}")
print(f"\nTrain shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")


Missing values BEFORE imputation:

Train set:
Administrative             195
Administrative_Duration    195
Informational              195
Informational_Duration     195
ProductRelated             195
ProductRelated_Duration    195
BounceRates                195
ExitRates                  195
PageValues                 195
SpecialDay                 195
Month                      195
OperatingSystems           195
Browser                    195
Region                     195
TrafficType                195
VisitorType                195
Weekend                    195
dtype: int64

Test set:
Administrative             52
Administrative_Duration    52
Informational              52
Informational_Duration     52
ProductRelated             52
ProductRelated_Duration    52
BounceRates                52
ExitRates                  52
PageValues                 52
SpecialDay                 52
Month                      52
OperatingSystems           52
Browser                    52
Region       

## 4. Feature Engineering

In [10]:
print("=" * 80)
print("COMPREHENSIVE FEATURE ENGINEERING")
print("=" * 80)

# Make copies for feature engineering
X_train_fe = X_train.copy()
X_test_fe = X_test.copy()

print(f"\nStarting features: {X_train_fe.shape[1]}")

# ===== 1. PAGE ENGAGEMENT FEATURES =====
print("\n1. Page Engagement Features:")

# Total pages visited
X_train_fe['Total_Pages'] = X_train_fe['Administrative'] + X_train_fe['Informational'] + X_train_fe['ProductRelated']
X_test_fe['Total_Pages'] = X_test_fe['Administrative'] + X_test_fe['Informational'] + X_test_fe['ProductRelated']
print("   ✓ Total_Pages: sum of all page types visited")

# Total time spent
X_train_fe['Total_Duration'] = X_train_fe['Administrative_Duration'] + X_train_fe['Informational_Duration'] + X_train_fe['ProductRelated_Duration']
X_test_fe['Total_Duration'] = X_test_fe['Administrative_Duration'] + X_test_fe['Informational_Duration'] + X_test_fe['ProductRelated_Duration']
print("   ✓ Total_Duration: sum of all time spent")

# Average time per page
X_train_fe['Avg_Time_Per_Page'] = X_train_fe['Total_Duration'] / (X_train_fe['Total_Pages'] + 1)  # +1 to avoid division by zero
X_test_fe['Avg_Time_Per_Page'] = X_test_fe['Total_Duration'] / (X_test_fe['Total_Pages'] + 1)
print("   ✓ Avg_Time_Per_Page: average time spent per page visit")

# Administrative engagement ratio
X_train_fe['Admin_Ratio'] = X_train_fe['Administrative'] / (X_train_fe['Total_Pages'] + 1)
X_test_fe['Admin_Ratio'] = X_test_fe['Administrative'] / (X_test_fe['Total_Pages'] + 1)
print("   ✓ Admin_Ratio: proportion of admin pages")

# Informational engagement ratio
X_train_fe['Info_Ratio'] = X_train_fe['Informational'] / (X_train_fe['Total_Pages'] + 1)
X_test_fe['Info_Ratio'] = X_test_fe['Informational'] / (X_test_fe['Total_Pages'] + 1)
print("   ✓ Info_Ratio: proportion of informational pages")

# Product engagement ratio
X_train_fe['Product_Ratio'] = X_train_fe['ProductRelated'] / (X_train_fe['Total_Pages'] + 1)
X_test_fe['Product_Ratio'] = X_test_fe['ProductRelated'] / (X_test_fe['Total_Pages'] + 1)
print("   ✓ Product_Ratio: proportion of product pages")

# Time spent ratios
X_train_fe['Admin_Time_Ratio'] = X_train_fe['Administrative_Duration'] / (X_train_fe['Total_Duration'] + 1)
X_test_fe['Admin_Time_Ratio'] = X_test_fe['Administrative_Duration'] / (X_test_fe['Total_Duration'] + 1)
print("   ✓ Admin_Time_Ratio: proportion of time on admin pages")

X_train_fe['Product_Time_Ratio'] = X_train_fe['ProductRelated_Duration'] / (X_train_fe['Total_Duration'] + 1)
X_test_fe['Product_Time_Ratio'] = X_test_fe['ProductRelated_Duration'] / (X_test_fe['Total_Duration'] + 1)
print("   ✓ Product_Time_Ratio: proportion of time on product pages")

# ===== 2. USER BEHAVIOR QUALITY FEATURES =====
print("\n2. User Behavior Quality Features:")

# Exit Rate vs Bounce Rate
X_train_fe['Exit_Bounce_Diff'] = X_train_fe['ExitRates'] - X_train_fe['BounceRates']
X_test_fe['Exit_Bounce_Diff'] = X_test_fe['ExitRates'] - X_test_fe['BounceRates']
print("   ✓ Exit_Bounce_Diff: difference between exit and bounce rates")

# Combined exit/bounce rate (user abandonment risk)
X_train_fe['Abandonment_Risk'] = (X_train_fe['ExitRates'] + X_train_fe['BounceRates']) / 2
X_test_fe['Abandonment_Risk'] = (X_test_fe['ExitRates'] + X_test_fe['BounceRates']) / 2
print("   ✓ Abandonment_Risk: combined exit and bounce rates")

# User quality score (inverse of bounce rate)
X_train_fe['User_Quality'] = 1 - X_train_fe['BounceRates']
X_test_fe['User_Quality'] = 1 - X_test_fe['BounceRates']
print("   ✓ User_Quality: inverse of bounce rate")

# ===== 3. MONETARY VALUE FEATURES =====
print("\n3. Monetary Value Features:")

# Revenue potential by pages
X_train_fe['Admin_Value_Per_Page'] = X_train_fe['PageValues'] / (X_train_fe['Administrative'] + 1)
X_test_fe['Admin_Value_Per_Page'] = X_test_fe['PageValues'] / (X_test_fe['Administrative'] + 1)
print("   ✓ Admin_Value_Per_Page: revenue potential per admin page")

X_train_fe['Product_Value_Per_Page'] = X_train_fe['PageValues'] / (X_train_fe['ProductRelated'] + 1)
X_test_fe['Product_Value_Per_Page'] = X_test_fe['PageValues'] / (X_test_fe['ProductRelated'] + 1)
print("   ✓ Product_Value_Per_Page: revenue potential per product page")

# Revenue per visit
X_train_fe['Revenue_Per_Visit'] = X_train_fe['PageValues'] / (X_train_fe['Total_Pages'] + 1)
X_test_fe['Revenue_Per_Visit'] = X_test_fe['PageValues'] / (X_test_fe['Total_Pages'] + 1)
print("   ✓ Revenue_Per_Visit: revenue per page visit")

# Revenue per time unit
X_train_fe['Revenue_Per_Minute'] = X_train_fe['PageValues'] / (X_train_fe['Total_Duration'] / 60 + 1)
X_test_fe['Revenue_Per_Minute'] = X_test_fe['PageValues'] / (X_test_fe['Total_Duration'] / 60 + 1)
print("   ✓ Revenue_Per_Minute: revenue efficiency by time")

# Has page value indicator
X_train_fe['Has_PageValue'] = (X_train_fe['PageValues'] > 0).astype(int)
X_test_fe['Has_PageValue'] = (X_test_fe['PageValues'] > 0).astype(int)
print("   ✓ Has_PageValue: binary indicator of any page value")

# ===== 4. TEMPORAL FEATURES =====
print("\n4. Temporal Features:")

# Special day impact
X_train_fe['Is_SpecialDay'] = (X_train_fe['SpecialDay'] > 0).astype(int)
X_test_fe['Is_SpecialDay'] = (X_test_fe['SpecialDay'] > 0).astype(int)
print("   ✓ Is_SpecialDay: binary indicator of special day event")

# Special day intensity
X_train_fe['SpecialDay_Intensity'] = X_train_fe['SpecialDay'] ** 2
X_test_fe['SpecialDay_Intensity'] = X_test_fe['SpecialDay'] ** 2
print("   ✓ SpecialDay_Intensity: quadratic special day feature")

# ===== 5. VISITOR TYPE FEATURES =====
print("\n5. Visitor Type Features:")

# Is returning visitor indicator
X_train_fe['Is_Returning'] = (X_train_fe['VisitorType'] == 'Returning_Visitor').astype(int)
X_test_fe['Is_Returning'] = (X_test_fe['VisitorType'] == 'Returning_Visitor').astype(int)
print("   ✓ Is_Returning: binary indicator of returning visitor")

# Is new visitor indicator
X_train_fe['Is_NewVisitor'] = (X_train_fe['VisitorType'] == 'New_Visitor').astype(int)
X_test_fe['Is_NewVisitor'] = (X_test_fe['VisitorType'] == 'New_Visitor').astype(int)
print("   ✓ Is_NewVisitor: binary indicator of new visitor")

# Is other visitor type (includes bot, other)
X_train_fe['Is_Other_Visitor'] = (X_train_fe['VisitorType'] != 'Returning_Visitor') & (X_train_fe['VisitorType'] != 'New_Visitor')
X_train_fe['Is_Other_Visitor'] = X_train_fe['Is_Other_Visitor'].astype(int)
X_test_fe['Is_Other_Visitor'] = (X_test_fe['VisitorType'] != 'Returning_Visitor') & (X_test_fe['VisitorType'] != 'New_Visitor')
X_test_fe['Is_Other_Visitor'] = X_test_fe['Is_Other_Visitor'].astype(int)
print("   ✓ Is_Other_Visitor: binary indicator of other visitor types")

# ===== 6. WEEKEND/WEEKDAY FEATURES =====
print("\n6. Temporal Patterns:")

# Is weekend indicator
X_train_fe['Is_Weekend'] = X_train_fe['Weekend'].astype(int)
X_test_fe['Is_Weekend'] = X_test_fe['Weekend'].astype(int)
print("   ✓ Is_Weekend: binary indicator (already categorical)")

# ===== 7. INTERACTION FEATURES =====
print("\n7. Interaction Features:")

# Product focus (product pages > info pages)
X_train_fe['Product_Focused'] = (X_train_fe['ProductRelated'] > X_train_fe['Informational']).astype(int)
X_test_fe['Product_Focused'] = (X_test_fe['ProductRelated'] > X_test_fe['Informational']).astype(int)
print("   ✓ Product_Focused: user focused more on products than info")

# Active visitor (high engagement)
X_train_fe['High_Engagement'] = (X_train_fe['Total_Duration'] > X_train_fe['Total_Duration'].median()).astype(int)
X_test_fe['High_Engagement'] = (X_test_fe['Total_Duration'] > X_test_fe['Total_Duration'].median()).astype(int)
print("   ✓ High_Engagement: above median engagement")

# Diverse visitor (visited multiple page types)
X_train_fe['Diverse_Visitor'] = ((X_train_fe['Administrative'] > 0).astype(int) + 
                                  (X_train_fe['Informational'] > 0).astype(int) + 
                                  (X_train_fe['ProductRelated'] > 0).astype(int))
X_test_fe['Diverse_Visitor'] = ((X_test_fe['Administrative'] > 0).astype(int) + 
                                 (X_test_fe['Informational'] > 0).astype(int) + 
                                 (X_test_fe['ProductRelated'] > 0).astype(int))
print("   ✓ Diverse_Visitor: count of distinct page types visited")

# Stability indicator (small difference between bounce and exit rates)
X_train_fe['Stable_User'] = (np.abs(X_train_fe['ExitRates'] - X_train_fe['BounceRates']) < 0.1).astype(int)
X_test_fe['Stable_User'] = (np.abs(X_test_fe['ExitRates'] - X_test_fe['BounceRates']) < 0.1).astype(int)
print("   ✓ Stable_User: exit and bounce rates are similar")

# ===== 8. DEVICE/BROWSER FEATURES =====
print("\n8. Device/Browser Features:")

# High OS count (many possible OSes, often bots)
X_train_fe['High_OS_Count'] = (X_train_fe['OperatingSystems'] > 5).astype(int)
X_test_fe['High_OS_Count'] = (X_test_fe['OperatingSystems'] > 5).astype(int)
print("   ✓ High_OS_Count: unusual OS patterns (potential bot indicator)")

# Common browser usage
X_train_fe['Browser_1_or_2'] = X_train_fe['Browser'].isin([1.0, 2.0]).astype(int)
X_test_fe['Browser_1_or_2'] = X_test_fe['Browser'].isin([1.0, 2.0]).astype(int)
print("   ✓ Browser_1_or_2: uses most common browsers")

# ===== 9. REGIONAL FEATURES =====
print("\n9. Regional Features:")

# Top regions (assuming regions 1-3 are high value)
X_train_fe['Top_Region'] = (X_train_fe['Region'] <= 3).astype(int)
X_test_fe['Top_Region'] = (X_test_fe['Region'] <= 3).astype(int)
print("   ✓ Top_Region: from high-volume regions")

# Traffic type classification
X_train_fe['Direct_Traffic'] = (X_train_fe['TrafficType'] == 1.0).astype(int)
X_test_fe['Direct_Traffic'] = (X_test_fe['TrafficType'] == 1.0).astype(int)
print("   ✓ Direct_Traffic: direct traffic indicator")

# ===== 10. COMPOSITE SCORING FEATURES =====
print("\n10. Composite Scoring Features:")

# Purchase likelihood score (higher product ratio + higher value + lower bounce)
X_train_fe['Purchase_Score'] = (X_train_fe['Product_Ratio'] * 3 + 
                                 X_train_fe['Revenue_Per_Visit'] * 2 + 
                                 (1 - X_train_fe['BounceRates']))
X_test_fe['Purchase_Score'] = (X_test_fe['Product_Ratio'] * 3 + 
                                X_test_fe['Revenue_Per_Visit'] * 2 + 
                                (1 - X_test_fe['BounceRates']))
print("   ✓ Purchase_Score: composite purchase likelihood")

# Engagement score
X_train_fe['Engagement_Score'] = (X_train_fe['Total_Duration'] / (X_train_fe['Total_Duration'].max() + 1) * 2 +
                                   X_train_fe['Total_Pages'] / (X_train_fe['Total_Pages'].max() + 1) +
                                   X_train_fe['User_Quality'])
X_test_fe['Engagement_Score'] = (X_test_fe['Total_Duration'] / (X_test_fe['Total_Duration'].max() + 1) * 2 +
                                  X_test_fe['Total_Pages'] / (X_test_fe['Total_Pages'].max() + 1) +
                                  X_test_fe['User_Quality'])
print("   ✓ Engagement_Score: composite engagement metric")

# Visitor value score
X_train_fe['Visitor_Value_Score'] = (X_train_fe['PageValues'] / (X_train_fe['PageValues'].max() + 1) * 3 +
                                      X_train_fe['Engagement_Score'])
X_test_fe['Visitor_Value_Score'] = (X_test_fe['PageValues'] / (X_test_fe['PageValues'].max() + 1) * 3 +
                                     X_test_fe['Engagement_Score'])
print("   ✓ Visitor_Value_Score: overall visitor value")

# ===== 11. POLYNOMIAL FEATURES (For Important Variables) =====
print("\n11. Polynomial Features:")

# Square of key metrics
X_train_fe['PageValues_Squared'] = X_train_fe['PageValues'] ** 2
X_test_fe['PageValues_Squared'] = X_test_fe['PageValues'] ** 2
print("   ✓ PageValues_Squared: polynomial expansion")

X_train_fe['ExitRates_Squared'] = X_train_fe['ExitRates'] ** 2
X_test_fe['ExitRates_Squared'] = X_test_fe['ExitRates'] ** 2
print("   ✓ ExitRates_Squared: polynomial expansion")

X_train_fe['BounceRates_Squared'] = X_train_fe['BounceRates'] ** 2
X_test_fe['BounceRates_Squared'] = X_test_fe['BounceRates'] ** 2
print("   ✓ BounceRates_Squared: polynomial expansion")

# ===== 12. OUTLIER DETECTION FEATURES =====
print("\n12. Outlier Detection Features:")

# Very high exit rate (unusual behavior)
X_train_fe['Very_High_ExitRate'] = (X_train_fe['ExitRates'] > 0.5).astype(int)
X_test_fe['Very_High_ExitRate'] = (X_test_fe['ExitRates'] > 0.5).astype(int)
print("   ✓ Very_High_ExitRate: indicates anomalous behavior")

# Perfect bounce rate (all bounces)
X_train_fe['Perfect_Bounce'] = (X_train_fe['BounceRates'] == 1.0).astype(int)
X_test_fe['Perfect_Bounce'] = (X_test_fe['BounceRates'] == 1.0).astype(int)
print("   ✓ Perfect_Bounce: all pages bounced")

# Zero engagement
X_train_fe['Zero_Engagement'] = ((X_train_fe['Total_Pages'] == 0) & (X_train_fe['Total_Duration'] == 0)).astype(int)
X_test_fe['Zero_Engagement'] = ((X_test_fe['Total_Pages'] == 0) & (X_test_fe['Total_Duration'] == 0)).astype(int)
print("   ✓ Zero_Engagement: no page visits or time spent")

# Update original datasets with engineered features
X_train = X_train_fe.copy()
X_test = X_test_fe.copy()

print(f"\n{'='*80}")
print(f"FEATURE ENGINEERING COMPLETE!")
print(f"{'='*80}")
print(f"New features created: {X_train.shape[1] - len(X_train_fe.columns) + len(X_train_fe.columns)}")
print(f"Total features now: {X_train.shape[1]}")
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")
print(f"\nNew feature columns:")
for col in X_train.columns:
    if col not in X_train_encoded.columns and col not in categorical_cols:
        print(f"  ✓ {col}")

COMPREHENSIVE FEATURE ENGINEERING

Starting features: 17

1. Page Engagement Features:
   ✓ Total_Pages: sum of all page types visited
   ✓ Total_Duration: sum of all time spent
   ✓ Avg_Time_Per_Page: average time spent per page visit
   ✓ Admin_Ratio: proportion of admin pages
   ✓ Info_Ratio: proportion of informational pages
   ✓ Product_Ratio: proportion of product pages
   ✓ Admin_Time_Ratio: proportion of time on admin pages
   ✓ Product_Time_Ratio: proportion of time on product pages

2. User Behavior Quality Features:
   ✓ Exit_Bounce_Diff: difference between exit and bounce rates
   ✓ Abandonment_Risk: combined exit and bounce rates
   ✓ User_Quality: inverse of bounce rate

3. Monetary Value Features:
   ✓ Admin_Value_Per_Page: revenue potential per admin page
   ✓ Product_Value_Per_Page: revenue potential per product page
   ✓ Revenue_Per_Visit: revenue per page visit
   ✓ Revenue_Per_Minute: revenue efficiency by time
   ✓ Has_PageValue: binary indicator of any page value


## 5. Encode Categorical Variables

In [11]:
# Encode categorical variables for tree-based models
X_train_processed = X_train.copy()
X_test_processed = X_test.copy()

print(f"{'='*60}")
print("Encoding categorical features with Label Encoding")
print(f"{'='*60}")
for col in categorical_cols:
    le = LabelEncoder()
    # Fit on combined train and test data to ensure consistency
    combined = pd.concat([X_train_processed[col], X_test_processed[col]], axis=0)
    le.fit(combined)
    
    X_train_processed[col] = le.transform(X_train_processed[col])
    X_test_processed[col] = le.transform(X_test_processed[col])
    label_encoders[col] = le

print(f"\n✓ Processed train shape: {X_train_processed.shape}")
print(f"✓ Processed test shape: {X_test_processed.shape}")


Encoding categorical features with Label Encoding

✓ Processed train shape: (9880, 56)
✓ Processed test shape: (2470, 56)


## 6. Define CV

In [12]:
# Cross-Validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Setup cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
# K-Fold XGBoost Models with Categorical Features (SAVE MODELS)
print("=" * 80)
print("K-FOLD XGBOOST WITH CATEGORICAL FEATURES - SAVE MODELS")
print("=" * 80)

xgboost_models = []
xgboost_cv_accuracies = []

print("\nTraining 5 XGBoost models with K-fold CV...\n")

fold_num = 1
for train_idx, val_idx in cv.split(X_train_processed, y_train):
    X_train_fold = X_train_processed.iloc[train_idx].copy()
    y_train_fold = y_train.iloc[train_idx]
    X_val_fold = X_train_processed.iloc[val_idx].copy()
    y_val_fold = y_train.iloc[val_idx]
    
    # ===== XGBOOST TRAINING =====
    # Train XGBoost
    xgb_model = XGBClassifier(
        n_estimators=500,
        max_depth=8,
        learning_rate=0.05,  
        subsample=0.8, 
        colsample_bytree=0.9,  # Column sampling per tree
        colsample_bylevel=0.9,  # Column sampling per (depth) level
        min_child_weight=10,  
        gamma=1,  
        reg_lambda=8,  
        reg_alpha=0,  
        random_state=42,
        verbosity=0,
        eval_metric='logloss',
        early_stopping_rounds=50,
        use_label_encoder=False,
    )
    
    xgb_model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        verbose=False
    )
    
    # Evaluate XGBoost on validation fold
    y_val_pred_xgb = xgb_model.predict(X_val_fold)
    fold_accuracy_xgb = accuracy_score(y_val_fold, y_val_pred_xgb)
    xgboost_cv_accuracies.append(fold_accuracy_xgb)
    xgboost_models.append(xgb_model)
    
    print(f"Fold {fold_num}: XGBoost Accuracy = {fold_accuracy_xgb:.6f}")
    fold_num += 1

print(f"\n{'='*80}")
print(f"K-Fold Summary:")
print(f"{'='*80}")
print(f"\nXGBoost:")
print(f"  Mean Accuracy: {np.mean(xgboost_cv_accuracies):.6f}")
print(f"  Std Accuracy:  {np.std(xgboost_cv_accuracies):.6f}")
print(f"  Models saved:  {len(xgboost_models)}")
print(f"{'='*80}")

# Extract best iterations from XGBoost fold models
print(f"\n{'='*80}")
print(f"EXTRACTING BEST ITERATIONS FROM XGBOOST K-FOLD MODELS")
print(f"{'='*80}\n")

best_iterations_xgb = []
for i, xgb_model in enumerate(xgboost_models):
    best_iter = xgb_model.best_iteration
    best_iterations_xgb.append(best_iter)
    print(f"Fold {i+1}: Best Iteration = {best_iter}")

avg_best_iteration_xgb = np.mean(best_iterations_xgb)
optimized_iterations_xgb = int(avg_best_iteration_xgb * 1.15)

print(f"\n{'='*80}")
print(f"Average Best Iteration: {avg_best_iteration_xgb:.2f}")
print(f"Optimized Iterations (avg × 1.15): {optimized_iterations_xgb}")
print(f"{'='*80}")

# Train full-dataset XGBoost model with optimized iterations
print(f"\nTraining XGBoost on FULL dataset with {optimized_iterations_xgb} iterations...\n")

xgboost_full_model = XGBClassifier(
    n_estimators=optimized_iterations_xgb,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    colsample_bylevel=0.9,
    min_child_weight=10,
    gamma=1,
    reg_lambda=8,
    reg_alpha=0,
    random_state=42,
    verbosity=0,
    use_label_encoder=False,
)

xgboost_full_model.fit(X_train_processed, y_train)

print(f"✓ XGBoost Full-dataset model trained successfully!")
print(f"  Total training samples: {len(X_train_processed)}")
print(f"  Iterations used: {optimized_iterations_xgb}")


K-FOLD XGBOOST WITH CATEGORICAL FEATURES - SAVE MODELS

Training 5 XGBoost models with K-fold CV...

Fold 1: XGBoost Accuracy = 0.906377
Fold 2: XGBoost Accuracy = 0.903846
Fold 3: XGBoost Accuracy = 0.895749
Fold 4: XGBoost Accuracy = 0.900810
Fold 5: XGBoost Accuracy = 0.906377

K-Fold Summary:

XGBoost:
  Mean Accuracy: 0.902632
  Std Accuracy:  0.004005
  Models saved:  5

EXTRACTING BEST ITERATIONS FROM XGBOOST K-FOLD MODELS

Fold 1: Best Iteration = 128
Fold 2: Best Iteration = 109
Fold 3: Best Iteration = 100
Fold 4: Best Iteration = 79
Fold 5: Best Iteration = 102

Average Best Iteration: 103.60
Optimized Iterations (avg × 1.2): 124

Training XGBoost on FULL dataset with 124 iterations...

✓ XGBoost Full-dataset model trained successfully!
  Total training samples: 9880
  Iterations used: 124


## 7. Create Submission File

In [15]:
# Create submission file with XGBoost predictions only
print("=" * 80)
print("CREATING SUBMISSION FILE - XGBOOST PREDICTIONS ONLY")
print("=" * 80)

# Get XGBoost predictions on test set
y_pred_xgb = xgboost_full_model.predict(X_test_processed)

print(f"\nXGBoost Prediction Distribution:")
xgb_class_0 = np.sum(y_pred_xgb == 0)
xgb_class_1 = np.sum(y_pred_xgb == 1)
print(f"  - Class 0 (No Revenue): {xgb_class_0} samples ({100*xgb_class_0/len(y_pred_xgb):.2f}%)")
print(f"  - Class 1 (Revenue):    {xgb_class_1} samples ({100*xgb_class_1/len(y_pred_xgb):.2f}%)")

# Create submission dataframe
submission_df = pd.DataFrame({
    'ID': test_ids,
    'Revenue': y_pred_xgb
})

# Save submission file
submission_df.to_csv('submission.csv', index=False)

print(f"\n✓ Submission file created successfully!")
print(f"\nFirst few rows of submission:")
print(submission_df.head(10))
print(f"\nSubmission shape: {submission_df.shape}")
print(f"File saved as: submission.csv")
print("=" * 80)


CREATING SUBMISSION FILE - XGBOOST PREDICTIONS ONLY

XGBoost Prediction Distribution:
  - Class 0 (No Revenue): 2183 samples (88.38%)
  - Class 1 (Revenue):    287 samples (11.62%)

✓ Submission file created successfully!

First few rows of submission:
   ID  Revenue
0   0        1
1   1        1
2   2        0
3   3        0
4   4        0
5   5        0
6   6        0
7   7        1
8   8        0
9   9        0

Submission shape: (2470, 2)
File saved as: submission.csv
